In [1]:
import numpy as np
import torch
import pandas as pd
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

C:\Users\Tatiana\anaconda3\envs\NlpEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tatiana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


***
The data file has four news articles, three of them are about the Tesla Roadster car in space and the fourth is about a different topic (a gas company). I want to check the similarity of the four articles using cosine similarity and Eulcidean distance using different vector representations of words. 
Will follow the article https://towardsdatascience.com/calculating-document-similarities-using-bert-and-other-models-b2c1a29c9630. 
The expectation is that the first three articles will be assessed as similar, while the fourth one different from them all.

A nice article on the interpretation of cosine similarity and Euclidean distance
https://www.baeldung.com/cs/euclidean-distance-vs-cosine-similarity
Cosine similarity is a metric used to measure how similar the documents are irrespective of their size. The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. 

***

In [2]:
data = pd.read_csv('data/roadster_news.csv', header=None)[0]

In [3]:
data

0    Электрокар Tesla Roadster Илона Маска, запущен...
1    Прошло почти четыре года с тех пор, как Илон М...
2    Tesla преодолела более трех миллиардов километ...
3    «Газпром» снизит поставки газа через «Северный...
Name: 0, dtype: object

***
First, let's compute cosine similarity and ED using the Tf-Idf matrix
***



In [4]:
vectorizer = TfidfVectorizer(stop_words=nltk.corpus.stopwords.words('russian'), 
                             token_pattern=r'\b[^\d\W]{4,20}\b') #"\b[a-zA-z]+'?[a-zA-Z]+'\b",
tfidf_mat = vectorizer.fit_transform(data)              


In [5]:
cosi = [] #cosine similarity
ed = [] #euclidean distance
for r1, r2 in itertools.combinations(range(tfidf_mat.shape[0]), 2):
    c = np.dot(tfidf_mat[r1], tfidf_mat[r2].T).toarray()[0][0]
    d = np.sqrt((tfidf_mat[r2] - tfidf_mat[r1]).power(2).sum())
    cosi.append((r1, r2, c))
    ed.append((r1, r2, d))
#
#ed = euclidean_distances(tfidf_mat)
#cosine_similarity(tfidf_mat)

In [7]:
cosi.sort(key=lambda v:v[2], reverse=True)
print(f'Most similar texts are {cosi[0][0]} and {cosi[0][1]} (cosine similarity is {cosi[0][2]} ):')
print(data[cosi[0][0]])
print('-')
print(data[cosi[0][1]])

Most similar texts are 1 and 2 (cosine similarity is 0.34236235938013404 ):
Прошло почти четыре года с тех пор, как Илон Маск запустил свой Tesla Roadster в космос на ракете Falcon Heavy компании SpaceX. Сейчас электромобиль находится на таком расстоянии от Земли, что Марс ему куда ближе родной планеты. Вскоре после запуска электрокара в космос был запущен и сайт Where is Roadster? для отслеживания его перемещения с помощью данных NASA. Ресурс указывает, что Roadster удаляется от Земли со скоростью 6 005 км/ч, в то время как к Марсу он движется со скоростью 27 955 км/ч. До Красной планеты осталось менее 320 миллионов км, а общий "пробег" машины почти достиг 3 млрд километров. Этого достаточно, чтобы проехать по всем дорогам мира 49,5 раз. В беседе CNN с астрономом Джонатаном Макдауэллом из Гарвард-Смитсоновского центра астрофизики выяснилось, что Tesla, вероятно, всё ещё цела, но могла быть повреждена ударами метеоритов. Интересно, что астрономы не наблюдали электромобиль в свои телеск

In [8]:
print(f'Least similar texts are {cosi[-1][0]} and {cosi[-1][1]} (cosine similarity is {cosi[-1][2]} ):')
print(data[cosi[-1][0]])
print('-')
print(data[cosi[-1][1]])

Least similar texts are 0 and 3 (cosine similarity is 0.00434486639530668 ):
Электрокар Tesla Roadster Илона Маска, запущенный в сторону Марса в начале 2018 года, постепенно разрушается в открытом космосе. К таким выводам пришли эксперты издания LiveScience. По их данным, автомобиль уже полностью лишился окраски кузова, кожаных сидений и покрышек.По словам химика из университета Индианы и эксперта по пластмассам и органическим молекулам Уильяма Кэрролла, в будущем от машины останутся лишь каркас и стекла. На молекулы распадутся и все пластиковые детали электрокара. Месяц назад машину уже официально признали космическим мусором — автомобиль был внесен в соответствующий каталог всех космических объектов искусственного происхождения (GCAT). Напомним, электрокар Tesla Roadster первого поколения, принадлежащий главе компаний Tesla и SpaceX Илону Маску, отправили в космос на ракете-носителе Falcon Heavy. Машина выступила в качестве балластной нагрузки, призванной продемонстрировать грузоподъ

In [9]:
ed.sort(key=lambda v:v[2])
print(f'The smallest distance between {ed[0][0]} and {ed[0][1]} (distance is {ed[0][2]} ):')
print(data[ed[0][0]])
print('-')
print(data[ed[0][1]])

The smallest distance between 1 and 2 (distance is 1.1468545161613708 ):
Прошло почти четыре года с тех пор, как Илон Маск запустил свой Tesla Roadster в космос на ракете Falcon Heavy компании SpaceX. Сейчас электромобиль находится на таком расстоянии от Земли, что Марс ему куда ближе родной планеты. Вскоре после запуска электрокара в космос был запущен и сайт Where is Roadster? для отслеживания его перемещения с помощью данных NASA. Ресурс указывает, что Roadster удаляется от Земли со скоростью 6 005 км/ч, в то время как к Марсу он движется со скоростью 27 955 км/ч. До Красной планеты осталось менее 320 миллионов км, а общий "пробег" машины почти достиг 3 млрд километров. Этого достаточно, чтобы проехать по всем дорогам мира 49,5 раз. В беседе CNN с астрономом Джонатаном Макдауэллом из Гарвард-Смитсоновского центра астрофизики выяснилось, что Tesla, вероятно, всё ещё цела, но могла быть повреждена ударами метеоритов. Интересно, что астрономы не наблюдали электромобиль в свои телескопы

In [10]:
print(f'Biggest distance between texts {ed[-1][0]} and {ed[-1][1]} (distance is {ed[-1][2]} ):')
print(data[ed[-1][0]])
print('-')
print(data[ed[-1][1]])

Biggest distance between texts 0 and 3 (distance is 1.4111379334456944 ):
Электрокар Tesla Roadster Илона Маска, запущенный в сторону Марса в начале 2018 года, постепенно разрушается в открытом космосе. К таким выводам пришли эксперты издания LiveScience. По их данным, автомобиль уже полностью лишился окраски кузова, кожаных сидений и покрышек.По словам химика из университета Индианы и эксперта по пластмассам и органическим молекулам Уильяма Кэрролла, в будущем от машины останутся лишь каркас и стекла. На молекулы распадутся и все пластиковые детали электрокара. Месяц назад машину уже официально признали космическим мусором — автомобиль был внесен в соответствующий каталог всех космических объектов искусственного происхождения (GCAT). Напомним, электрокар Tesla Roadster первого поколения, принадлежащий главе компаний Tesla и SpaceX Илону Маску, отправили в космос на ракете-носителе Falcon Heavy. Машина выступила в качестве балластной нагрузки, призванной продемонстрировать грузоподъемн

***
Now let's try using GloVe word embeddings. For simplicity, we will consider each document as one sentence and work with doc vectors. Because I use articles written in Russian, I use word embeddings from Navec (https://natasha.github.io/navec/) that were trained using news articles. 
***

In [11]:
from navec import Navec
nv = Navec.load('data/embeddings/navec_news_v1_1B_250K_300d_100q.tar')

In [12]:
tokens = list(map(vectorizer.build_tokenizer(),data))
#min_token_len = 3
tokens = [[t.lower() for t in doc_toks if t in vectorizer.vocabulary_] for doc_toks in tokens]

***
Extract from the article:
Now we have to represent every document as a single vector. We can either average or sum over every word vector and convert every 64X300 representation into a 300-dimensional representation. But averaging or summing over all the words would lose the semantic and contextual meaning of the documents. Different lengths of the documents would also have an adverse effect on such operations.

One better way of doing this could be taking a weighted average of word vectors using the tf-idf weights. This can handle the variable length problem to a certain extent but cannot keep the semantic and contextual meaning of words. After doing that we can use the pairwise distances to calculate similar documents as we did in the tf-idf model.
***

In [13]:
from sklearn.preprocessing import normalize

In [14]:
emb_sz = nv.pq.dim
tfidf_df = pd.DataFrame(tfidf_mat.toarray())
docs_emb = np.zeros((len(data), emb_sz))
for i in range(len(data)):
    for t in tokens[i]:
        if t in nv.vocab:
            docs_emb[i] += nv[t] * tfidf_df[vectorizer.vocabulary_[t]][i]

In [22]:
docs_emb_norm = normalize(docs_emb, axis=1, norm='l2')
cosi = [] #cosine similarity
ed = [] #euclidean distance
for r1, r2 in itertools.combinations(range(docs_emb.shape[0]), 2):
    c = np.dot(docs_emb_norm[r1], docs_emb_norm[r2].T)
    d = np.sqrt(np.power(docs_emb[r2] - docs_emb[r1], 2).sum())
    cosi.append((r1, r2, c))
    ed.append((r1, r2, d))
#cosine_similarity(docs_emb)
#euclidean_distances(docs_emb)

In [23]:
cosi.sort(key=lambda v:v[2], reverse=True)
print(f'Most similar texts are {cosi[0][0]} and {cosi[0][1]} (cosine similarity is {cosi[0][2]} ):')
print(f'Least similar texts are {cosi[-1][0]} and {cosi[-1][1]} (cosine similarity is {cosi[-1][2]} ):')
print(f'The smallest distance between {ed[0][0]} and {ed[0][1]} (distance is {ed[0][2]} ):')
print(f'Biggest distance between texts {ed[-1][0]} and {ed[-1][1]} (distance is {ed[-1][2]} ):')

Most similar texts are 1 and 2 (cosine similarity is 0.822083303190346 ):
Least similar texts are 1 and 3 (cosine similarity is 0.4568089726027239 ):
The smallest distance between 0 and 1 (distance is 10.304522040145553 ):
Biggest distance between texts 2 and 3 (distance is 17.376559808309153 ):
